# Using a consolidated metadata file to rapidly access MUR SST v4.1

The consolidated metadata file only has to be created once:
Step 1 is [here](https://github.com/cgentemann/cloud_science/blob/master/zarr_meta/cloud_mur_v41-all-step1.ipynb)
Step 2 is [here](https://github.com/cgentemann/cloud_science/blob/master/zarr_meta/cloud_mur_v41-all-step2.ipynb)

NASA JPL PODAAC has put the entire [MUR SST](https://podaac.jpl.nasa.gov/dataset/MUR-JPL-L4-GLOB-v4.1) dataset on AWS cloud as individual netCDF files, **but all ~7000 of them are netCDF files.**\ Accessing one file works well, but accessing multiple files is **very slow** because the metadata for each file has to be queried. Here, we create **fast access** by consolidating the metadata and accessing the entire dataset rapidly via zarr. More background on this project:
[medium article](https://medium.com/pangeo/fake-it-until-you-make-it-reading-goes-netcdf4-data-on-aws-s3-as-zarr-for-rapid-data-access-61e33f8fe685) and in this [repo](https://github.com/lsterzinger/fsspec-reference-maker-tutorial). We need help developing documentation and more test datasets. If you want to help, we are working in the [Pangeo Gitter](https://gitter.im/pangeo-data/cloud-performant-netcdf4).


To run this code:
- you need to set up your `.netrc` file in your home directory with your earthdata login info


Authors:
- [Chelle Gentemann](https://github.com/cgentemann)
- [Rich Signell](https://github.com/rsignell-usgs)
- [Lucas Steringzer](https://github.com/lsterzinger/)
- [Martin Durant](https://github.com/martindurant)

Credit:
- Funding: Interagency Implementation and Advanced Concepts Team [IMPACT](https://earthdata.nasa.gov/esds/impact) for the Earth Science Data Systems (ESDS) program
- AWS Public Dataset [Program](https://registry.opendata.aws/mur/)
- [QuanSight](https://www.quansight.com/) for creating Qhub, [ESIP Labs ](https://www.esipfed.org/lab) for deploying it, and [AWS Sustainablity](https://aws.amazon.com/government-education/sustainability-research-credits/) for funding it!

In [ ]:
import s3fs
import requests
from urllib import request
import xarray as xr
import fsspec
import hvplot.xarray

In [ ]:
json_consolidated = 's3://esip-qhub-public/nasa/mur/murv41_consolidated_20211025.json'

In [ ]:
from earthdata import Auth #, DataColletions, DataGranules, Accessor
auth = Auth().login()

In [ ]:
def begin_s3_direct_access():
    url="https://archive.podaac.earthdata.nasa.gov/s3credentials"
    response = requests.get(url).json()
    return s3fs.S3FileSystem(key=response['accessKeyId'],
                             secret=response['secretAccessKey'],
                             token=response['sessionToken'],
                             client_kwargs={'region_name':'us-west-2'})


In [ ]:
url="https://archive.podaac.earthdata.nasa.gov/s3credentials"
response = requests.get(url).json()

- Consolidated metadata test (without simple_templates)

In [ ]:
%%time

rpath = json_consolidated
print(rpath)
s_opts = {'requester_pays':True, 'skip_instance_cache':True}
r_opts = {'key':response['accessKeyId'],
          'secret':response['secretAccessKey'],
          'token':response['sessionToken'],
          'client_kwargs':{'region_name':'us-west-2'}}

fs = fsspec.filesystem("reference", 
                       fo=rpath, 
                       ref_storage_args=s_opts,
                       remote_protocol='s3', 
                       remote_options=r_opts)#,simple_templates=True)
m = fs.get_mapper("")
ds = xr.open_dataset(m, decode_times=False, engine="zarr", consolidated=False)
ds.close()
ds

In [ ]:
#test getting a random value
ds['analysed_sst'].sel(time='2005-12-20 12:00', lat=0,lon=0,method='nearest')

In [ ]:
%%time
dy = ds['analysed_sst'].sel(time='2002-12-20', method='nearest')
dy.hvplot.quadmesh(x='lon', y='lat', geo=True, rasterize=True, cmap='turbo' )